In [ ]:
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="f053486a0d364fe09c5bddf49fc4657e",
                                                           client_secret="701cacd9264046d7889ab46f6f7610a9"))


In [ ]:
f = open("Playlist_properties/playlist_ids.txt" ,"r")

playlist_list = []
for line in f:
    playlist_id, x, y = line.split() 
    playlist_list.append(playlist_id)
f.close()

f = open("Playlist_properties/playlist_names.txt" ,"r")
dict_ = open("Pickle_Files/Msd_id_to_spotify_id.pkl", "rb")
dict_ = pickle.load(dict_)
dict_ = dict(map(reversed, dict_.items()))
f.close()

print(len(playlist_list))



In [ ]:
main_dict = {}
for i in range(len(playlist_list)):
    playlist_id = playlist_list[i]
    offset = 0
    count = 0
    song_ids = set()
    while(1):
        results1 = sp.playlist_items(playlist_id, offset = offset, limit = 100)
        if(len(results1['items']) == 0):
            break
        for j in range(len(results1['items'])):
            try:
                song_id = results1['items'][j]['track']['id']
                if(song_id in dict_):
                    count+=1
                    song_ids.add(song_id)
            except:
                continue
        offset += len(results1['items'])
    song_ids = list(song_ids)
    
    for song_id in song_ids:
        if song_id not in main_dict:
            main_dict[song_id] = []
        main_dict[song_id].append(i)

# print(main_dict)


In [ ]:
print(len(main_dict))

In [ ]:
f = open("Pickle_Files/songs_to_playlist_mapping.pkl", "wb")
pickle.dump(main_dict, f)
f.close()

In [ ]:
import pandas as pd
df = pd.read_csv("Dataset/unlabelled_song_features_and_lda_topics_3.csv")

new_column = []
for i in df.index:
    song_id = df['sp_song_id'][i]
    new_column.append(main_dict[song_id])

df["Playlists"] = new_column

new_column = []
for i in df.index:
    release_date = df['release_date'][i]
    try:
        release_date = int(release_date[-4:])
    except:
        release_date = int(release_date[:4])
    new_column.append(release_date)

df['release_date'] = new_column

df.to_csv("Dataset/updated_dataset_3.csv", index = False)


In [ ]:
df.head()